In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [49]:
import os
from __future__ import division, print_function
HOMEPATH = "/home/ubuntu/fastai/"
print("HOMEPATH:", HOMEPATH)
DATA_PATH = '/home/ubuntu/fastai/data/text_prediction/'
print("DATA_PATH:", DATA_PATH)
MODEL_PATH = '/home/ubuntu/fastai/data/text_prediction/models/'
print("MODEL_PATH:", DATA_PATH)


HOMEPATH: /home/ubuntu/fastai/
DATA_PATH: /home/ubuntu/fastai/data/text_prediction/
MODEL_PATH: /home/ubuntu/fastai/data/text_prediction/


In [4]:
os.chdir(HOMEPATH)
print ("current working directory:", os.getcwd())

%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

current working directory: /home/ubuntu/fastai


Using Theano backend.


In [5]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

#https://keras.io/layers/wrappers/
#https://keras.io/layers/recurrent/#simplernn
#https://keras.io/activations/


## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [6]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600901


In [7]:
!tail {path} -n25

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [9]:
#path = 'data/wiki/'
#text = open(path+'small.txt').read().lower()
#print('corpus length:', len(text))

#text = text[0:1000000]

In [11]:
print('corpus length:', len(text))
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)
print ("chars:", chars)

corpus length: 600901
total chars: 60
chars: ['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x86', '\xa4', '\xa6', '\xa9', '\xab', '\xc3']


In [12]:
chars.insert(0, "\0")

In [13]:
''.join(chars[1:-6])
#display all the chars except last 6.

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz'

In [14]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [19]:
print ("char_indices:", type(char_indices), len(char_indices))
print ("char_indices:", char_indices)


print ("indices_char:", type(indices_char), len(indices_char))
print ("indices_char:", indices_char)

char_indices: <type 'dict'> 60
char_indices: {'\x00': 0, '\x86': 54, '\n': 1, '\xa9': 57, '!': 3, ' ': 2, '"': 4, '\xa4': 55, "'": 5, '\xa6': 56, ')': 7, '(': 6, '\xab': 58, '-': 9, ',': 8, '.': 10, '1': 12, '0': 11, '3': 14, '2': 13, '5': 16, '4': 15, '7': 18, '6': 17, '9': 20, '8': 19, ';': 22, ':': 21, '=': 23, '?': 24, '\xc3': 59, '[': 25, ']': 26, '_': 27, 'a': 28, 'c': 30, 'b': 29, 'e': 32, 'd': 31, 'g': 34, 'f': 33, 'i': 36, 'h': 35, 'k': 38, 'j': 37, 'm': 40, 'l': 39, 'o': 42, 'n': 41, 'q': 44, 'p': 43, 's': 46, 'r': 45, 'u': 48, 't': 47, 'w': 50, 'v': 49, 'y': 52, 'x': 51, 'z': 53}
indices_char: <type 'dict'> 60
indices_char: {0: '\x00', 1: '\n', 2: ' ', 3: '!', 4: '"', 5: "'", 6: '(', 7: ')', 8: ',', 9: '-', 10: '.', 11: '0', 12: '1', 13: '2', 14: '3', 15: '4', 16: '5', 17: '6', 18: '7', 19: '8', 20: '9', 21: ':', 22: ';', 23: '=', 24: '?', 25: '[', 26: ']', 27: '_', 28: 'a', 29: 'b', 30: 'c', 31: 'd', 32: 'e', 33: 'f', 34: 'g', 35: 'h', 36: 'i', 37: 'j', 38: 'k', 39: 'l', 40

In [20]:
idx = [char_indices[c] for c in text]

In [21]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [22]:
''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## Preprocess and create model

In [23]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 600862


In [24]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [25]:
sentences.shape, next_chars.shape

((600860, 40), (600860, 40))

In [26]:
n_fac = 24

In [27]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [28]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

## Train

In [29]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [30]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1495s - loss: 1.5022  


In [31]:
print_example()

ethics is a basic foundation of all that curiosity place itself and the herods of life is their logical
beasts (to severe that nther being a god of the
world is anything finally
hailed the work of
the emancipation with a definite man, the freely and to the decided the
constant new differently and art, he has not would taken others, to us and
experienced away


In [32]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1493s - loss: 1.2824  


In [33]:
print_example()

ethics is a basic foundation of all that appears
races and imperativeness in this conclusions and requirements would once the naked."--they
nature it is only perhaps does not obtrust
his
assertions merely "near_ they are called point: almost his conditions of pale,
cats) one
who never denires a conception of the great laboriousness. the philosopher enjoys a 


In [34]:
model.optimizer.lr=0.001

In [35]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1514s - loss: 1.2483  


In [36]:
print_example()

ethics is a basic foundation of all that appears to this problem of the conscience, there are universal ways,
as possible, it has the english old conception
of all the discoverers in jeality; one wishes to be engended; but all nature for being good-natured, and does not surer of the
strength and every something was
always nature with regard to the art (may
e


In [37]:
model.optimizer.lr=0.0001

In [38]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 2591s - loss: 1.2288  


In [39]:
print_example()

ethics is a basic foundation of all that one of a man has still might be homozed.

214. how different with the same
signify from
life, because us always become more for
such a "thing-in-itself." this consciousness of the excitation of the complete civilizations. the koon 'stirst of ultimate love. indeed, it will carry us do the very remote as surviving ctill


In [40]:
model.save_weights(MODEL_PATH+'char_rnn.h5')

In [41]:
model.optimizer.lr=0.00001

In [42]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1653s - loss: 1.2144  


In [43]:
print_example()

ethics is a basic foundation of all that the future would no longer know what still be called "morality," is not sensible to dispense themselves they were finally shame in every philosophy; what? practicism
and depth--what has not different than a different world as in the religious latter can no longer prevails one wish to be called upon our
pathos in the
g


In [44]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 1491s - loss: 1.2019  


In [45]:
print_example()

ethics is a basic foundation of all that we could
be a new german historical
value and
affair. like their existing experiences; it now becomes better yet subjection, by experience that has gort so dreadful would fain attain
what is a dangerous
and manner begins to be
understood in one's
soul;
the good end--if
it has example to sew knowledge with a christian 


In [46]:
print_example()

ethics is a basic foundation of all that only this noble soul was accasionally developed thereby. he who would no longer appear mankind for the seriousness:--and however,
who becomes conventional being, as creatively only a bad act "a thinker flaughter, and we make use of satisfaction is not enough to appear, or consequence, and
privileges of religion from t


In [47]:
model.save_weights(MODEL_PATH+'char_rnn_2.h5')

In [ ]:
print_example()